# import libraries

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from flask import Flask, request, jsonify
from keras.preprocessing import image
from keras.models import load_model
from keras.models import save_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# process class for dir

In [2]:
def load_data(data_dir):
    images = []
    labels = []

    for filename in os.listdir(os.path.join(data_dir, "images")):
        if filename.endswith('.jpg'):
            img_path = os.path.join(data_dir, "images", filename)
            label_path = os.path.join(data_dir, "labels", filename.replace('.jpg', '.txt'))

#             print(f"Processing {filename}...")  # Print filename for debugging

            if os.path.isfile(label_path):
                with open(label_path, 'r') as label_file:
                    line = label_file.readline().strip()

                    if not line:
                        print(f"Empty line in file: {label_path}")
                        continue

                    try:
                        label = int(line.split()[0])
                        labels.append(label)
                    except (ValueError, IndexError) as e:
                        print(f"Error processing line in file {label_path}: {line}")
                        print(f"Error details: {e}")

                # Resize the image to the expected input shape (224, 224)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))
                images.append(img)

    return np.array(images), np.array(labels)


# Specify your dataset directory
dataset_dir = '/kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train'

# Load the data
images, labels = load_data(dataset_dir)

Empty line in file: /kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train/labels/volume_17_slice_118_jpg.rf.59848b5e32900276089b16eda778dcca.txt
Empty line in file: /kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train/labels/volume_251_slice_59_jpg.rf.5d403476e7b51ad5aa550cdbe3a7d796.txt
Empty line in file: /kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train/labels/volume_51_slice_138_jpg.rf.3dd10a466ac2f2e7a762ce4f2a0bfd2c.txt
Empty line in file: /kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train/labels/volume_22_slice_57_jpg.rf.30e20221893edf3fb8e5de85567c5d27.txt
Empty line in file: /kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train/labels/volume_232_slice_98_jpg.rf.a9f52374f1363f05c627e39af3f4879a.txt
Empty line in file: /kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/train/labels/vol

# Split the dataset into training and testing sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# sorted labels and categories

In [4]:
# Ensure labels start from 0
min_label = np.min(y_train)
y_train = y_train - min_label
y_test = y_test - min_label

# Count the number of unique labels
num_classes = len(np.unique(y_train))

# Convert labels to categorical format
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

# build model

In [5]:
# Load the pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)  # Additional Dense Layer
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model  (extra layer of 256)
model.fit(X_train, y_train_categorical, epochs=15, batch_size=32, validation_data=(X_test, y_test_categorical))

# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test_categorical)[1]
print(f'Test Accuracy: {accuracy}')

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/15
172/172 [==============================] - 73s 200ms/step - loss: 1.5174 - accuracy: 0.6115 - val_loss: 1.2368 - val_accuracy: 0.6280
Epoch 2/15
172/172 [==============================] - 31s 183ms/step - loss: 0.6343 - accuracy: 0.6595 - val_loss: 0.9470 - val_accuracy: 0.6193
Epoch 3/15
172/172 [==============================] - 31s 183ms/step - loss: 0.5626 - accuracy: 0.7219 - val_loss: 0.5321 - val_accuracy: 0.7600
Epoch 4/15
172/172 [==============================] - 31s 182ms/step - loss: 0.4945 - accuracy: 0.7710 - val_loss: 0.8905 - val_accuracy: 0.6156
Epoch 5/15
172/172 [==============================] - 31s 182ms/step - loss: 0.4439 - accuracy: 0.8022 - val_loss: 0.9062 - val_accuracy: 0.3931
Epoch 6/15
172/172 [==============================] - 31s 183ms/step - loss: 0.5045 - accuracy: 0.7695 - val_loss: 3.3419 - val_accuracy: 0.6484
Epoch 7/15
172/172 [==============================] - 31s 183ms/s

In [6]:
# Preprocess the data (e.g., normalization)
X_train = X_train / 255.0
X_test = X_test / 255.0

# Ensure labels start from 0
min_label = np.min(y_train)
y_train = y_train - min_label
y_test = y_test - min_label

# Count the number of unique labels
num_classes = len(np.unique(y_train))

# Convert labels to categorical format
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

# Split the data into training and validation sets
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(X_train, y_train_categorical, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))  # Additional Dense Layer
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=15, batch_size=32, validation_data=(X_val, y_val_encoded))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Epoch 1/15
137/137 [==============================] - 11s 52ms/step - loss: 0.6915 - accuracy: 0.6061 - val_loss: 0.7331 - val_accuracy: 0.6086
Epoch 2/15
137/137 [==============================] - 5s 36ms/step - loss: 0.6683 - accuracy: 0.6380 - val_loss: 0.8922 - val_accuracy: 0.6040
Epoch 3/15
137/137 [==============================] - 5s 36ms/step - loss: 0.6474 - accuracy: 0.6332 - val_loss: 0.6716 - val_accuracy: 0.6241
Epoch 4/15
137/137 [==============================] - 5s 36ms/step - loss: 0.6255 - accuracy: 0.6693 - val_loss: 0.6842 - val_accuracy: 0.6195
Epoch 5/15
137/137 [==============================] - 5s 36ms/step - loss: 0.6219 - accuracy: 0.6635 - val_loss: 0.6639 - val_accuracy: 0.6496
Epoch 6/15
137/137 [==============================] - 5s 36ms/step - loss: 0.6005 - accuracy: 0.6816 - val_loss: 0.7279 - val_accuracy: 0.6451
Epoch 7/15
137/137 [==============================] - 5s 36ms/step - loss: 0.5916 - accuracy: 0.6880 - val_loss: 0.7842 - val_accuracy: 0.667

In [7]:
#save keras model
model.save(r'model_512_256_epoch15.keras')